In [1]:
# %load_ext lab_black

# Standard Imports

In [1]:
import sys

# sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append("../build/mechsystem")

from mass_spring import *

from pythreejs import *
from time import sleep

## Double Pendulum to chanin

In [2]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

f0 = mss.add(Fix((0, 0, 0)))  # Fixed anchor
m1 = mss.add(Mass(1.0, (0.0, 0.0, -1.0)))  # First mass
mss.add(Spring(1.0, 50000, (f0, m1)))  # First chain link
# Double pendulum
m2 = mss.add(Mass(1.0, (0.5, 0.0, -2.0)))
m3 = mss.add(Mass(1.0, (1.0, 0.0, -3.0)))
mss.add(Spring(1.0, 40000, (m1, m2)))
mss.add(Spring(1.0, 40000, (m2, m3)))

2

In [ ]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(
            SphereBufferGeometry(0.15, 16, 16),
            MeshStandardMaterial(color="red"),
            position=m.pos,
        )
    )

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(
            SphereBufferGeometry(0.2, 16, 16),
            MeshStandardMaterial(color="blue"),
            position=f.pos,
        )
    )

def get_spring_segments():
    segs = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        segs.append([pA, pB])
    return segs


spring_geo = LineSegmentsGeometry(positions=get_spring_segments())
spring_mat = LineMaterial(linewidth=3, color="cyan")
springs = LineSegments2(spring_geo, spring_mat)

### Definiton of the View

In [7]:
def create_scene(masses, fixes, springs, width=600, height=400):
    axes = AxesHelper(1)
    camera = PerspectiveCamera(
        position=[0, 10, 0],
        aspect=width / height,
        lookAt=[0, 0, 0],
        up=[0, 0, 1],
    )
    key_light = DirectionalLight(position=[0, 10, 10])
    ambient_light = AmbientLight()
    scene = Scene(
        children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light]
    )
    controller = OrbitControls(controlling=camera)
    renderer = Renderer(
        camera=camera,
        scene=scene,
        controls=[controller],
        width=width,
        height=height,
    )
    return renderer

In [8]:
create_scene(masses, fixes, springs)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(0.0, 10.0, 0.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0,…

In [9]:
for _ in range(20000):
    mss.simulate(0.02, 100)
    for m, vis in zip(mss.masses, masses):
        vis.position = (m.pos[0], m.pos[1], m.pos[2])

    spring_geo = LineSegmentsGeometry(positions=get_spring_segments())
    springs.geometry = spring_geo

    sleep(0.01)

KeyboardInterrupt: 